This is December 2021 Tabulat Playground series

### Outline:
0. Load libraries and custom functions.
1. Load data.
2. Preliminary data analysis: explore features and a target, delete unneeded features, create new features.
3. Train-test split.
4. Missing values. In some cases it may be useful to explore skew and perform log-transform before imputing missing values.
5. Feature engineering. Transform skewed variables, do OHC and scaling.
6. Fit models.
7. Evaluate models.
8. Feature importance, error analysis. Based on the results, go to 2. and iterate.
9. Make predictions.

In [3]:
import pynvml
pynvml.nvmlInit()
deviceCount = pynvml.nvmlDeviceGetCount()
for i in range(deviceCount):
    handle = pynvml.nvmlDeviceGetHandleByIndex(i)
    print(f"Device {i} {pynvml.nvmlDeviceGetName(handle).decode()}")

Device 0 Tesla P100-PCIE-16GB


In [13]:
# 0. Load libraries #

import numpy as np
import pandas as pd
import os, time, warnings, optuna, gc
import seaborn as sns
import matplotlib.pyplot as plt
import tensorflow as tf

from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer, make_column_transformer
from sklearn.svm import SVC, SVR
from sklearn.preprocessing import LabelBinarizer, LabelEncoder, OrdinalEncoder, OneHotEncoder, StandardScaler
from sklearn.linear_model import LogisticRegression, LinearRegression, Lasso, Ridge
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV, train_test_split, KFold
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier, VotingClassifier
from sklearn.metrics import accuracy_score, f1_score, r2_score, mean_squared_error, make_scorer
from sklearn.inspection import permutation_importance
from scipy.special import inv_boxcox
from xgboost import XGBClassifier, XGBRegressor
import lightgbm as lgb

pd.set_option('display.max_columns', 20)
pd.set_option('mode.chained_assignment', None)
pd.set_option('display.expand_frame_repr', False)
warnings.filterwarnings('ignore')

def draw_histograms(df, variables, n_rows, n_cols):
    # stolen from https://stackoverflow.com/questions/29530355/plotting-multiple-histograms-in-grid
    fig=plt.figure()
    for i, var_name in enumerate(variables):
        ax=fig.add_subplot(n_rows,n_cols,i+1)
        df[var_name].hist(bins=10,ax=ax)
        ax.set_title(var_name+" Distribution")
    fig.tight_layout()  
    plt.show()


def fillna_mp_i1(df_train, df_test, df_pred, num_features, cat_features, num_fill='median', cat_fill='mode'):
    """This function speeds up filling missing values for 3 main datasets using different imputation methods.
    Later may replace it with some subclass.
    Example: 
    fillna_mp_i1(X_train, X_test, X_pred, num_cols, cat_cols)"""
    
    # set df_pred to None if it does not exist
    if (cat_features is not None):
        if (cat_fill=='mode'):

            df_train[cat_features] = df_train[cat_features].fillna(value=df_train[cat_features].mode().iloc[0])
            df_test[cat_features] = df_test[cat_features].fillna(value=df_train[cat_features].mode().iloc[0])
            if (df_pred is not None):
                df_pred[cat_features] = df_pred[cat_features].fillna(value=df_train[cat_features].mode().iloc[0])

        if (cat_fill=='missing'):

            df_train[cat_features] = df_train[cat_features].fillna(value='missing')
            df_test[cat_features] = df_test[cat_features].fillna(value='missing')
            if (df_pred is not None):
                df_pred[cat_features] = df_pred[cat_features].fillna(value='missing')
        
    if (num_fill=='median'):
        df_train[num_features] = df_train[num_features].fillna(value=df_train[num_features].median())
        df_test[num_features] = df_test[num_features].fillna(value=df_train[num_features].median())
        if (df_pred is not None):
            df_pred[num_features] = df_pred[num_features].fillna(value=df_train[num_features].median())    
    
    if (cat_features is not None):
        all_good = (
        (np.prod(df_train[num_features+cat_features].shape)==df_train[num_features+cat_features].count().sum()) and 
        (np.prod(df_test[num_features+cat_features].shape) == df_test[num_features+cat_features].count().sum()))
        if (all_good):
            print('Missing values imputed successfully')
        else:
            print('There are still some missing values...')
    else:
        all_good = (
        (np.prod(df_train[num_features].shape)==df_train[num_features].count().sum()) and 
        (np.prod(df_test[num_features].shape) == df_test[num_features].count().sum()))
        if (all_good):
            print('Missing values imputed successfully')
        else:
            print('There are still some missing values...')
# END

    
def add_misDummy_mp_i1(df_train, df_test, df_pred, features):
    """This function creates new dummy columns for missing features.
    Example: add_misDummy_mp_i1(X_train, X_test, X_pred, ['Age'])"""
    # set df_pred to None if it does not exist
    
    columns_before = df_train.shape[1]
    
    for feature_name in features:
        
        if df_train[feature_name].count()==df_train.shape[0]:
            continue
        
        misColName = 'mis'+feature_name
        df_train.loc[df_train[feature_name].isnull(), misColName]=1
        df_train.loc[df_train[feature_name].notnull(), misColName]=0
        df_test.loc[df_test[feature_name].isnull(), misColName]=1
        df_test.loc[df_test[feature_name].notnull(), misColName]=0
        if (df_pred is not None):
            df_pred.loc[df_pred[feature_name].isnull(), misColName]=1
            df_pred.loc[df_pred[feature_name].notnull(), misColName]=0
            
        columns_after = df_train.shape[1]
            
    print(columns_after-columns_before, ' dummy features added')
# END
   

def discretize_mp_i1(df_train, df_test, df_pred, feature, ntiles, delete_feature=False):
    """This function divides a continuous feature into quantile groups.
    Example: discretize_mp_i1(X_train, X_test, X_pred, 'Age', 15)"""
    # set df_pred to None if it does not exist
    _,bin = pd.qcut(df_train[feature], ntiles, retbins = True, labels = False, duplicates = 'drop')
    df_train[feature+'Ntile'] = pd.cut(df_train[feature], labels=False, duplicates = 'drop', bins = bin ,include_lowest = True)
    df_test[feature+'Ntile'] = pd.cut(df_test[feature], labels=False, duplicates = 'drop', bins = bin ,include_lowest = True)
    if (df_pred is not None):
        df_pred[feature+'Ntile'] = pd.cut(df_pred[feature], labels=False, duplicates = 'drop', bins = bin ,include_lowest = True)
    if (delete_feature==True):
        df_train.drop(columns=[feature], inplace=True)
        df_test.drop(columns=[feature], inplace=True)
        df_pred.drop(columns=[feature], inplace=True)
    print('Discretized ',feature, ' into ', len(bin)-1, ' bins')
# END


def log_transformer_mp_i1(df_train, df_test, df_pred=None, feature_subset=False, max_skew=3):
    """This function divides a continuous feature into quantile groups.
    Example: log_transformer_mp_i1(X_train, X_test, X_pred, feature_subset=num_cols)"""
    # set df_pred to None if it does not exist
    if (feature_subset==False):
        features_totransform = df_train.columns
    else:
        features_totransform = feature_subset.copy()
    skewed_vars = list(df_train.skew()[(df_train.skew())>max_skew].index)
    for col in list(set(skewed_vars)&set(features_totransform)):
        df_train[col] = np.log1p(df_train[col])
        df_test[col] = np.log1p(df_test[col])
        if (df_pred is not None):
            df_pred[col] = np.log1p(df_pred[col])
    print('Skewed columns log-transformed: ', list(set(skewed_vars)&set(features_totransform)))
# END
    
    
def add_dummyfeatures(df_train, df_test, df_pred, feature_dict):
    """This function adds dummy feature when some feature is equal to value, specified in a dictionary.
    Example: add_dummyfeatures(X_train, X_test, X_pred, {'RoomService':0, 'Spa':0, 'VRDeck':0, 'ShoppingMall':0})"""
    input_dimensions = np.array([df_train.shape[1], df_test.shape[1], df_pred.shape[1]])
    for i in range(len(list(feature_dict.items()))):
        feature,value = list(feature_dict.keys())[i], list(feature_dict.values())[i]
        df_train.loc[df_train[feature]==value,(str(feature)+str(value))]=1
        df_train.loc[df_train[feature]!=value,(str(feature)+str(value))]=0
        df_test.loc[df_test[feature]==value,(str(feature)+str(value))]=1
        df_test.loc[df_test[feature]!=value,(str(feature)+str(value))]=0
        df_pred.loc[df_pred[feature]==value,(str(feature)+str(value))]=1
        df_pred.loc[df_pred[feature]!=value,(str(feature)+str(value))]=0
    output_dimensions = np.array([df_train.shape[1], df_test.shape[1], df_pred.shape[1]])
    print(output_dimensions-input_dimensions, ' variables created') 
# END



time0 = time.time()


#1. Load data #

df = pd.read_csv('../input/tabular-playground-series-dec-2021/train.csv')
pred = pd.read_csv('../input/tabular-playground-series-dec-2021/test.csv')
pred0 = pred.copy()
print(df.shape, pred.shape)

# 2. pEDA #

df = df.sample(200000, random_state=2)

df.drop(columns = ['Id'], inplace = True)
pred.drop(columns = ['Id'], inplace = True)
print(df.Cover_Type.value_counts())
#df.head()

#[[col, df[col].nunique()] for col in df.columns]
#df.count()
df.Cover_Type.value_counts()
#df.skew()

# 3. Train-test split #

train_y = df[['Cover_Type']]
train_y.replace([1, 2, 3, 4, 6, 7], [0,1,2,3,4,5], inplace = True)
train_x = df.drop(columns = ['Cover_Type'])

X_train, X_test, y_train, y_test = train_test_split(train_x, train_y, test_size = 0.05, random_state = 1)

print(X_train.shape, X_test.shape, y_train.shape, pred.shape)


# 4. Missing values #

#X_train.count()

#X_train.skew()

# 5. Feature engineering #

#log_transformer_mp_i1(X_train, X_test, pred)
#X_train.skew()
# all skewed variables are already ohc-ed, so their skew does not matter.

ss = StandardScaler()

for col in X_train.columns:
    X_train[[col]] = ss.fit_transform(X_train[[col]])
    X_test[[col]] = ss.transform(X_test[[col]])
    pred[[col]] = ss.transform(pred[[col]])
    
gc.collect()

(4000000, 56) (1000000, 55)
2    112641
1     74007
3      9804
7      2986
6       537
4        25
Name: Cover_Type, dtype: int64
(190000, 54) (10000, 54) (190000, 1) (1000000, 55)


118

In [14]:
# 6. Model fitting #

f1w = make_scorer(f1_score , average='weighted')

time1 = time.time()
lr = LogisticRegression()
param_grid = {'C':[1, 10, 100]}
lrm = GridSearchCV(lr, param_grid, cv=2, scoring=f1w)
lrm.fit(X_train, y_train)
print('Logistic', lrm.best_params_, lrm.best_score_, time.time()-time1)

Logistic {'C': 100} 0.913665263032583 48.924094676971436


In [15]:
# Fit XGBoost using Optuna

time1 = time.time()

def objective(trial, n_splits=2, n_jobs=-1, early_stopping_rounds=50):
    params = {
        "tree_method": 'gpu_hist',
        "gpu_id": 0,
        "verbosity": 0,  # 0 (silent) - 3 (debug)
        "n_estimators": 500,
        "max_depth": trial.suggest_int("max_depth", 2, 10),
        "learning_rate": trial.suggest_uniform("learning_rate", 0.02, 0.3),
        "colsample_bytree": trial.suggest_uniform("colsample_bytree", 0.2, 1),
        "subsample": trial.suggest_uniform("subsample", 0.3, 1),
        "alpha": trial.suggest_loguniform("alpha", 0.001, 10.0),
        "lambda": trial.suggest_loguniform("lambda", 1e-8, 10.0),
        "gamma": trial.suggest_loguniform("gamma", 1e-8, 10.0),
        "min_child_weight": trial.suggest_loguniform("min_child_weight", 0.1, 200),
        "n_jobs": n_jobs,
    }

    X = X_train
    y = y_train
    
    model = XGBClassifier(**params)
    rkf = KFold(n_splits=n_splits)
    X_values = X.values
    y_values = y.values
    y_pred = np.zeros_like(y_values)
    for train_index, test_index in rkf.split(X_values):
        X_A, X_B = X_values[train_index, :], X_values[test_index, :]
        y_A, y_B = y_values[train_index], y_values[test_index]
        model.fit(X_A, y_A, eval_set=[(X_B, y_B)],
                  early_stopping_rounds=early_stopping_rounds, verbose = False)
        y_pred[test_index] += model.predict(X_B).reshape(-1,1)
    return (f1_score(y_train, y_pred, average='weighted'))

time1 = time.time()
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=50)
print('Total time ', time.time()-time1)
hp = study.best_params
for key, value in hp.items():
    print(f"{key:>20s} : {value}")
print(f"{'best objective value':>20s} : {study.best_value}")

optuna_hyperpars = study.best_params
optuna_hyperpars['tree_method']='gpu_hist'
optuna_hyperpars['gpu_id']=0
optuna_hyperpars['n_estimators']=300
#optuna_hyperpars
optuna_xgb = XGBClassifier(**optuna_hyperpars)
optuna_xgb.fit(X_train, y_train)

[I 2022-06-05 20:33:27,205] A new study created in memory with name: no-name-a9755574-f713-41ef-bd1e-ee0a92fd656e
[I 2022-06-05 20:33:46,631] Trial 0 finished with value: 0.9534767545048135 and parameters: {'max_depth': 7, 'learning_rate': 0.21113722551112757, 'colsample_bytree': 0.3881817595458446, 'subsample': 0.8992526390885487, 'alpha': 0.001535245335376505, 'lambda': 1.9252771105793597e-07, 'gamma': 2.783201019807266e-08, 'min_child_weight': 2.2352379352482075}. Best is trial 0 with value: 0.9534767545048135.
[I 2022-06-05 20:34:02,537] Trial 1 finished with value: 0.9534240422814795 and parameters: {'max_depth': 9, 'learning_rate': 0.2592985787854512, 'colsample_bytree': 0.8847198282339721, 'subsample': 0.929956252160677, 'alpha': 5.189437529672751, 'lambda': 1.2162035137454986e-08, 'gamma': 1.8976802726507724e-08, 'min_child_weight': 1.2957745949902637}. Best is trial 0 with value: 0.9534767545048135.
[I 2022-06-05 20:34:15,614] Trial 2 finished with value: 0.9314161500214528 an

Total time  739.828848361969
           max_depth : 5
       learning_rate : 0.20346585707435932
    colsample_bytree : 0.8329198208930636
           subsample : 0.9294125465901408
               alpha : 0.09316233522489825
              lambda : 1.2028182990743631e-08
               gamma : 0.00039686315852934033
    min_child_weight : 3.79832958178436
best objective value : 0.9552847403759469


XGBClassifier(alpha=0.09316233522489825, base_score=0.5, booster='gbtree',
              callbacks=None, colsample_bylevel=1, colsample_bynode=1,
              colsample_bytree=0.8329198208930636, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None,
              gamma=0.00039686315852934033, gpu_id=0, grow_policy='depthwise',
              importance_type=None, interaction_constraints='',
              lambda=1.2028182990743631e-08, learning_rate=0.20346585707435932,
              max_bin=256, max_cat_to_onehot=4, max_delta_step=0, max_depth=5,
              max_leaves=0, min_child_weight=3.79832958178436, missing=nan,
              monotone_constraints='()', n_estimators=300, n_jobs=0,
              num_parallel_tree=1, objective='multi:softprob', predictor='auto', ...)

In [ ]:
#xgb = XGBClassifier(tree_method = 'gpu_hist', gpu_id = 0)
#param_grid = {'eta':[0.1, 0.2, 0.3], 'max_depth':[4,6,8], 'n_estimators':[300]}
#xgbm = GridSearchCV(xgb, param_grid, cv=2, scoring=f1w, verbose=1)


#xgbm.fit(X_train, y_train)
#print('XGB', xgbm.best_params_, xgbm.best_score_, time.time()-time1)

In [ ]:
#Visualize parameter importances.
optuna.visualization.plot_param_importances(study)

In [ ]:
optuna.visualization.plot_slice(study)

# 

In [16]:
# 7. Model evaluation #

print('Logistic IS', f1_score(y_train, lrm.predict(X_train), average='weighted'), 
      f1_score(y_test, lrm.predict(X_test), average='weighted'))

#print('XGB_gs IS', f1_score(y_train, xgbm.predict(X_train), average='weighted'), 
#      f1_score(y_test, xgbm.predict(X_test), average='weighted'))

print('XGB_Optuna IS', f1_score(y_train, optuna_xgb.predict(X_train), average='weighted'), 
      f1_score(y_test, optuna_xgb.predict(X_test), average='weighted'))


Logistic IS 0.9142852115986049 0.9135316469635705
XGB_Optuna IS 0.9768323995480069 0.9576501379878262


In [18]:
pred

,Id,Elevation,Aspect,Slope,Horizontal_Distance_To_Hydrology,Vertical_Distance_To_Hydrology,Horizontal_Distance_To_Roadways,Hillshade_9am,Hillshade_Noon,Hillshade_3pm,...,Soil_Type31,Soil_Type32,Soil_Type33,Soil_Type34,Soil_Type35,Soil_Type36,Soil_Type37,Soil_Type38,Soil_Type39,Soil_Type40
0,4000000,-0.757296,-0.669686,0.569859,0.467441,0.526933,1.012499,0.199197,-0.358207,1.243558,...,-0.169528,-0.19819,-0.197285,-0.11282,-0.126101,-0.104283,-0.112389,-0.20703,-0.203746,-0.178611
1,4000001,-0.539119,0.012576,-0.481125,-0.031308,-0.186873,-1.119912,0.231767,0.762779,0.169396,...,-0.169528,-0.19819,-0.197285,-0.11282,-0.126101,-0.104283,-0.112389,-0.20703,-0.203746,-0.178611
2,4000002,-0.116617,-0.860719,0.453083,-0.949360,-0.114036,-0.697074,-0.321920,-0.178850,0.512214,...,-0.169528,-0.19819,5.068816,-0.11282,-0.126101,-0.104283,-0.112389,-0.20703,-0.203746,-0.178611
3,4000003,-0.192806,-0.296716,-1.065006,-0.499161,1.197037,0.277656,0.720314,0.852458,0.032269,...,-0.169528,-0.19819,-0.197285,-0.11282,-0.126101,-0.104283,-0.112389,-0.20703,-0.203746,-0.178611
4,4000004,-1.010104,-1.288270,-1.298558,-1.028807,0.818283,1.380679,0.036348,0.000508,2.020611,...,-0.169528,-0.19819,-0.197285,-0.11282,-0.126101,-0.104283,-0.112389,-0.20703,-0.203746,-0.178611
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
999995,4999995,-0.026576,0.931356,-0.014021,-0.552126,0.701743,-0.870915,0.329476,1.076656,-0.539094,...,-0.169528,-0.19819,-0.197285,-0.11282,-0.126101,-0.104283,-0.112389,-0.20703,-0.203746,-0.178611
999996,4999996,0.686828,-0.542330,0.102755,-0.786053,-0.070333,1.495309,-0.093932,-0.896281,0.032269,...,-0.169528,-0.19819,-0.197285,-0.11282,-0.126101,-0.104283,-0.112389,-0.20703,4.908068,-0.178611
999997,4999997,-1.030883,-0.915300,0.569859,-1.046461,1.036795,-0.613569,-0.419630,0.179866,0.100832,...,-0.169528,-0.19819,-0.197285,-0.11282,7.930134,-0.104283,-0.112389,-0.20703,-0.203746,-0.178611
999998,4999998,1.133571,-0.797041,-0.364349,-0.331441,0.599770,0.570682,0.459756,-1.254997,-1.521838,...,-0.169528,-0.19819,-0.197285,-0.11282,-0.126101,-0.104283,-0.112389,-0.20703,-0.203746,-0.178611


In [33]:
temp = optuna_xgb.predict(pred)
temp = pd.DataFrame(temp, columns = ['Cover_Type'])
print(temp.head())
temp.replace([0,1,2,3,4,5], [1, 2, 3, 4, 6, 7], inplace = True)
print(temp.head())

   Cover_Type
0           1
1           1
2           1
3           1
4           1
   Cover_Type
0           2
1           2
2           2
3           2
4           2


In [35]:

#yhat = optuna_xgb.predict(pred)

submission_df_xgb = pd.DataFrame({'Id': id.Id, 'Cover_Type': temp['Cover_Type']}, columns=['Id', 'Cover_Type'])
#submission_df_bt.Transported = np.array([bool(x) for x in submission_df_bt.Transported])
submission_df_xgb.to_csv('KP14_xgb.csv',index=False)

os.chdir(r'/kaggle/working')

from IPython.display import FileLink
FileLink(r'KP14_xgb.csv')

/kaggle/working/KP14_xgb.csv